In [12]:
data = {
    "lampada_I": [2.40, 2.52, 2.65, 2.77, 2.89, 3.02, 3.12, 3.23, 3.34, 3.43, 3.52, 3.65, 3.76], # A
    "lampada_V": [4.40, 4.89, 5.36, 5.8, 6.26, 6.79, 7.21, 7.72, 8.18, 8.65, 9.09, 9.59, 10.07], # V
    "sensor_V": [3.5, 4.6, 5.7, 6.7, 7.8, 9.0, 10, 11.3, 12.4, 13.4, 14.9, 16.5, 18], # V10e-3
}

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(data)

In [15]:
df.columns

Index(['lampada_I', 'lampada_V', 'sensor_V'], dtype='object')

In [16]:
df = df[['lampada_V', 'lampada_I', 'sensor_V']]

In [18]:
df.to_csv("dados_aula.txt", sep=" ", index=False, header=False)